# ChatGPT OpenAI Find Cash Flowing Real Estate Properties
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1VLqM6bc1iTuqfLPak_GjbJAjjJ7CrtPS?usp=sharing)

## Overview
| Detail Tag            | Information                                                                                        |
|-----------------------|----------------------------------------------------------------------------------------------------|
| Originally Created By | Ariel Herrera arielherrera@analyticsariel.com |
| External References   | Open AI API |
| Input Datasets        | Source name |
| Output Datasets       | Source name |
| Input Data Source     | Pandas DataFrame |
| Output Data Source    | String |

## History
| Date         | Developed By  | Reason                                                |
|--------------|---------------|-------------------------------------------------------|
| 13th May 2023 | Ariel Herrera | Create notebook. |

## Getting Started
1. Copy this notebook -> File -> Save a Copy in Drive
2. Directions

## Useful Resources
- [Google Colab Cheat Sheet](https://towardsdatascience.com/cheat-sheet-for-google-colab-63853778c093)

## <font color="blue">Install Packages</font>

In [35]:
!pip install -q langchain openai

## <font color="blue">Imports</font>

In [36]:
from google.colab import files
from getpass import getpass
import io
import pandas as pd
import plotly.express as px
import sqlite3
from sqlalchemy import create_engine
from sqlalchemy.pool import StaticPool
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain
from langchain.agents import create_pandas_dataframe_agent

pd.set_option("display.max_columns", None)

In [37]:
# open ai key
openai_api_key = getpass('Enter the open ai key: ')

Enter the open ai key: ··········


## <font color="blue">Data</font>


In [38]:
# upload file
uploaded = files.upload()

Saving Birmingham-Listings.csv to Birmingham-Listings (1).csv


In [39]:
# transform file into pandas dataframe
df = pd.read_csv(io.BytesIO(uploaded[list(uploaded.keys())[0]]))
print('Number of rows:', len(df))
print('Number of columns:', len(df.columns))
df.head()

Number of rows: 81
Number of columns: 33


,Street,City,State,ZIP Code,Image,Price,House Type,Year Built,Sq. Footage,Lot Size,Bedrooms,Bathrooms,Description,Down Payment %,Interest Rate %,Loan Years,Loan Amount,Payment Months,Closing Costs %,Total Renovation Cost,Rent,Other Revenue,Mortgage Payment,HOA Fees,Insurance,Property Taxes,Vacancy Rate Allocation,Management Fee,Maintenance,Other Costs,Initial Costs,Monthly Profit,Cash on Cash
0,106 Avenue V,Birmingham,AL,35214,https://photos.zillowstatic.com/fp/92879571d0d...,120000,Single Family,1950,1225,6098,4,2,This property has 2 houses in the same lot. Yo...,0.2,0.07000,30,96000.0,360,0.03,0,1435,0,638.69,0,35,65.000,120,132,26,0,"27,600.00",418.31,18.19
1,2004 Henry Crumpton Dr SW,Birmingham,AL,35211,https://photos.zillowstatic.com/fp/edd9c35b0a2...,129900,Single Family,1950,1572,6098,3,2,"This charming newly renovated 3 bedroom, 2 bat...",0.2,0.06384,30,103920.0,360,0.03,0,1470,0,648.94,0,38,70.363,123,135,27,0,"29,877.00",427.70,17.18
2,1520 Matt Leonard Dr SW,Birmingham,AL,35211,https://photos.zillowstatic.com/fp/5660344d253...,135000,Single Family,1963,1598,8712,3,2,Come check out this wonderful brick and spacio...,0.2,0.06384,30,108000.0,360,0.03,0,1320,0,674.42,0,39,73.125,110,121,24,0,"31,050.00",278.46,10.76
3,1212 Haven Dr,Birmingham,AL,35214,https://photos.zillowstatic.com/fp/cd4932b2e74...,110000,Single Family,1938,1650,0,2,2,Nice Level 2BR 2BA home with basement worksho...,0.2,0.07000,30,88000.0,360,0.03,0,1091,0,585.47,0,32,59.583,91,100,20,0,"25,300.00",202.95,9.63
4,1012 Inzer Dr,Birmingham,AL,35214,https://photos.zillowstatic.com/fp/6c56ae445a0...,119600,Single Family,1959,3193,0.89 Acres,3,2,Brick home on dead end street. Living room/di...,0.2,0.07000,30,95680.0,360,0.03,0,1184,0,636.56,0,35,64.783,99,108,22,0,"27,508.00",218.16,9.52


In [40]:
# features
# add an address field
df['Address'] = df.apply(lambda x: x['Street'] + ', ' + x['City'] + ', ' + x['State'] + ' ' + str(x['ZIP Code']), axis=1)
df.head(2)

,Street,City,State,ZIP Code,Image,Price,House Type,Year Built,Sq. Footage,Lot Size,Bedrooms,Bathrooms,Description,Down Payment %,Interest Rate %,Loan Years,Loan Amount,Payment Months,Closing Costs %,Total Renovation Cost,Rent,Other Revenue,Mortgage Payment,HOA Fees,Insurance,Property Taxes,Vacancy Rate Allocation,Management Fee,Maintenance,Other Costs,Initial Costs,Monthly Profit,Cash on Cash,Address
0,106 Avenue V,Birmingham,AL,35214,https://photos.zillowstatic.com/fp/92879571d0d...,120000,Single Family,1950,1225,6098,4,2,This property has 2 houses in the same lot. Yo...,0.2,0.07000,30,96000.0,360,0.03,0,1435,0,638.69,0,35,65.000,120,132,26,0,"27,600.00",418.31,18.19,"106 Avenue V, Birmingham, AL 35214"
1,2004 Henry Crumpton Dr SW,Birmingham,AL,35211,https://photos.zillowstatic.com/fp/edd9c35b0a2...,129900,Single Family,1950,1572,6098,3,2,"This charming newly renovated 3 bedroom, 2 bat...",0.2,0.06384,30,103920.0,360,0.03,0,1470,0,648.94,0,38,70.363,123,135,27,0,"29,877.00",427.70,17.18,"2004 Henry Crumpton Dr SW, Birmingham, AL 35211"


In [41]:
# describe dataset
df.describe().apply(lambda x: x.apply('{0:.2f}'.format))

,ZIP Code,Price,Year Built,Sq. Footage,Bedrooms,Bathrooms,Down Payment %,Interest Rate %,Loan Years,Loan Amount,Payment Months,Closing Costs %,Total Renovation Cost,Rent,Other Revenue,HOA Fees,Insurance,Property Taxes,Vacancy Rate Allocation,Management Fee,Maintenance,Other Costs,Cash on Cash
count,81.00,81.00,81.00,81.00,81.00,81.00,81.00,81.00,81.00,81.00,81.00,81.00,81.00,81.00,81.00,81.00,81.00,81.00,81.00,81.00,81.00,81.00,81.00
mean,35212.73,246421.60,1978.43,1701.90,3.35,2.19,0.20,0.07,30.00,197137.28,360.00,0.03,0.00,1432.23,0.00,3.27,71.88,133.48,119.38,131.31,26.30,0.00,-4.56
std,1.31,106881.75,36.17,516.48,0.59,0.74,0.00,0.00,0.00,85505.40,0.00,0.00,0.00,569.27,0.00,13.48,31.12,57.89,47.38,52.21,10.51,0.00,9.01
min,35211.00,104000.00,1920.00,820.00,2.00,1.00,0.20,0.06,30.00,83200.00,360.00,0.03,0.00,0.00,0.00,0.00,30.00,56.33,0.00,0.00,0.00,0.00,-32.22
25%,35211.00,149900.00,1950.00,1288.00,3.00,2.00,0.20,0.06,30.00,119920.00,360.00,0.03,0.00,1091.00,0.00,0.00,44.00,81.20,91.00,100.00,20.00,0.00,-8.98
50%,35212.00,239900.00,1965.00,1660.00,3.00,2.00,0.20,0.07,30.00,191920.00,360.00,0.03,0.00,1302.00,0.00,0.00,70.00,129.95,109.00,119.00,24.00,0.00,-5.75
75%,35214.00,329900.00,2023.00,1958.00,4.00,3.00,0.20,0.07,30.00,263920.00,360.00,0.03,0.00,1721.00,0.00,0.00,96.00,178.70,143.00,158.00,32.00,0.00,0.20
max,35214.00,528000.00,2023.00,3305.00,5.00,5.00,0.20,0.07,30.00,422400.00,360.00,0.03,0.00,2936.00,0.00,108.00,154.00,286.00,245.00,269.00,54.00,0.00,18.19


In [42]:
# view distribution on CoCR
fig = px.histogram(df, x="Cash on Cash")
fig.show()

## <font color="blue">Open AI</font>


In [43]:
# create pandas agent
# full docs - https://python.langchain.com/en/latest/modules/agents/toolkits/examples/pandas.html
agent = create_pandas_dataframe_agent(OpenAI(temperature=0, openai_api_key=openai_api_key, max_tokens=300), df, verbose=True)

### Question #1 - Which address has the highest cash on cash return?

In [44]:
agent.run('which address has the highest cash on cash')



> Entering new AgentExecutor chain...
Thought: I need to find the address with the highest cash on cash
Action: python_repl_ast
Action Input: df['Cash on Cash'].max()
Observation: 18.19
Thought: I need to find the address associated with this cash on cash
Action: python_repl_ast
Action Input: df.loc[df['Cash on Cash'] == df['Cash on Cash'].max()]['Address'].iloc[0]
Observation: 106 Avenue V, Birmingham, AL 35214
Thought: I now know the final answer
Final Answer: The address with the highest cash on cash is 106 Avenue V, Birmingham, AL 35214.

> Finished chain.


'The address with the highest cash on cash is 106 Avenue V, Birmingham, AL 35214.'

In [45]:
idx = 0 # view information on a single row
print('Full address:', df.iloc[idx, :]['Address'])
print('Description:', df.iloc[idx, :]['Description'])
print('Initial Costs: ${}'.format(df.iloc[idx, :]['Initial Costs']))
print('Monthly Profit: ${}'.format(df.iloc[idx, :]['Monthly Profit']))
print('Cash on Cash Return: ${}'.format(df.iloc[idx, :]['Cash on Cash']))

Full address: 106 Avenue V, Birmingham, AL 35214
Description: This property has 2 houses in the same lot. You can live in one while renting the other. Or use one of them as "in law suite." This property has many use possibilities.  Come see it today and choose yours! Property will be sold "as is"
Initial Costs: $27,600.00
Monthly Profit: $418.31
Cash on Cash Return: $18.19


### Question #2 - How many addresses have a positive cash flow?

In [46]:
agent.run('how many addresses have a positive monthly profit')



> Entering new AgentExecutor chain...
Thought: I need to calculate the monthly profit for each address
Action: python_repl_ast
Action Input: df['Monthly Profit'] = df['Rent'] + df['Other Revenue'] - df['Mortgage Payment'] - df['HOA Fees'] - df['Insurance'] - df['Property Taxes'] - df['Vacancy Rate Allocation'] - df['Management Fee'] - df['Maintenance'] - df['Other Costs']
Observation: TypeError("unsupported operand type(s) for -: 'int' and 'str'")
Thought: I need to convert the columns to numeric
Action: python_repl_ast
Action Input: df['Mortgage Payment'] = pd.to_numeric(df['Mortgage Payment'])
df['HOA Fees'] = pd.to_numeric(df['HOA Fees'])
df['Insurance'] = pd.to_numeric(df['Insurance'])
df['Property Taxes'] = pd.to_numeric(df['Property Taxes'])
df['Vacancy Rate Allocation'] = pd.to_numeric(df['Vacancy Rate Allocation'])
df['Management Fee'] = pd.to_numeric(df['Management Fee'])
df['Maintenance'] = pd.to_numeric(df['Maintenance'])
df['Other Costs'] = pd.to_numeric(df['Other Costs']

KeyboardInterrupt: ignored

### Question #3 - How many addresses have a description that contains <font color="green">investment opportunity</font>?

In [47]:
agent.run('how many addresses have a description that contains investment opportunity')



> Entering new AgentExecutor chain...
Thought: I need to find the descriptions that contain the phrase "investment opportunity"
Action: python_repl_ast
Action Input: df[df['Description'].str.contains('investment opportunity')]['Address'].count()
Observation: 1
Thought: I now know the final answer
Final Answer: 1

> Finished chain.


'1'

In [49]:
df[df['Description'].str.contains('investment opportunity')]

,Street,City,State,ZIP Code,Image,Price,House Type,Year Built,Sq. Footage,Lot Size,Bedrooms,Bathrooms,Description,Down Payment %,Interest Rate %,Loan Years,Loan Amount,Payment Months,Closing Costs %,Total Renovation Cost,Rent,Other Revenue,Mortgage Payment,HOA Fees,Insurance,Property Taxes,Vacancy Rate Allocation,Management Fee,Maintenance,Other Costs,Initial Costs,Monthly Profit,Cash on Cash,Address
15,528 Dunkirk Dr,Birmingham,AL,35214,https://photos.zillowstatic.com/fp/3e3ad06c231...,137500,Single Family,1957,1492,0.39 Acres,3,1,This property is currently tenant-occupied and...,0.2,0.06384,30,110000.0,360,0.03,0,1094,0,686.90,0,40,74.479,91,100,20,0,"31,625.00",81.62,3.1,"528 Dunkirk Dr, Birmingham, AL 35214"


In [51]:
df.iloc[15,:]['Description']

'This property is currently tenant-occupied and has been well-maintained, making it the perfect investment opportunity. Great location with a new construction located just behind this house. Only 3 minutes from the grocery store and it is currently rented out for $1,050 through 09/30/2023. New tin roof and HVAC in Q4 of 2018.'

## <font color="blue">Want to get started in Real Estate Data Analytics?</font>
Checkout my course below 👇

[Introduction to Real Estate Data Analytics](https://analyticsariel.teachable.com/p/real-estate-data-analytics)


# End Notebook